In [ ]:
%conda install pandas -y
%conda install scikit-learn -y
%pip install streamlit

In [ ]:
import pandas as pd  

# Load datasets
diabetes = pd.read_csv("datasets/diabetes.csv")
heart = pd.read_csv("datasets/heart_disease.csv")
parkinsonsv2 = pd.read_csv("datasets/parkinsons.csv")

# Display first few rows
print("Diabetes Dataset:\n", diabetes.head(), "\n")
print("Heart Disease Dataset:\n", heart.head(), "\n")
print("parkinsons Disease Dataset:\n", parkinsons.head(), "\n")

# Check for missing values
print("Missing Values:\n")
print("Diabetes:\n", diabetes.isnull().sum(), "\n")
print("Heart:\n", heart.isnull().sum(), "\n")
print("parkinsons:\n", parkinsons.isnull().sum(), "\n")

# Drop non-feature columns


Diabetes Dataset:
    Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1   

Heart Disease Dataset:
    age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   0       145   233    1        2      150      0      2.3      2   
1   67    1   3       160   286    0        2      108      1      1.5      1   
2   67    1   3

In [16]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

def preprocess_data(df, target_column):
    X = df.drop(columns=[target_column])  # Features
    y = df[target_column]  # Target (labels)

    # Convert categorical columns to numeric using Label Encoding
    for col in X.columns:
        if X[col].dtype == 'object':  # If column is categorical
            X[col] = LabelEncoder().fit_transform(X[col])

    # Normalize numerical features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split into train & test sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test, scaler

# Run preprocessing again
X_train_diabetes, X_test_diabetes, y_train_diabetes, y_test_diabetes, scaler_diabetes = preprocess_data(diabetes, "Outcome")
X_train_heart, X_test_heart, y_train_heart, y_test_heart, scaler_heart = preprocess_data(heart, "target")
X_train_parkinsons, X_test_parkinsons, y_train_parkinsons, y_test_parkinsons, scaler_parkinsons = preprocess_data(parkinsons, "status")

print("Preprocessing complete!")


KeyError: "['status'] not found in axis"

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def train_and_evaluate(X_train, X_test, y_train, y_test, disease_name):
    # Initialize and train the model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{disease_name} Model Accuracy: {accuracy:.4f}")

    return model

# Train models for each dataset
model_diabetes = train_and_evaluate(X_train_diabetes, X_test_diabetes, y_train_diabetes, y_test_diabetes, "Diabetes")
model_heart = train_and_evaluate(X_train_heart, X_test_heart, y_train_heart, y_test_heart, "Heart Disease")
model_parkinsons = train_and_evaluate(X_train_parkinsons, X_test_parkinsons, y_train_parkinsons, y_test_parkinsons, "Parkinson's")

Diabetes Model Accuracy: 0.7532
Heart Disease Model Accuracy: 0.8852
Parkinson's Model Accuracy: 0.8974


In [9]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import joblib

def preprocess_data(df, target_column, scaler_path, encoder_path):
    X = df.drop(columns=[target_column])  # Features
    y = df[target_column]  # Target (labels)

    # Convert categorical columns to numeric using Label Encoding
    encoders = {}
    for col in X.columns:
        if X[col].dtype == 'object':  # If column is categorical
            encoders[col] = LabelEncoder()
            X[col] = encoders[col].fit_transform(X[col])

    # Normalize numerical features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split into train & test sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Save scaler and encoders
    joblib.dump(scaler, scaler_path)
    joblib.dump(encoders, encoder_path)

    return X_train, X_test, y_train, y_test

# Run preprocessing and save scalers
X_train_diabetes, X_test_diabetes, y_train_diabetes, y_test_diabetes = preprocess_data(
    diabetes, "Outcome", "Models/diabetes_scaler.pkl", "Models/diabetes_encoders.pkl"
)
X_train_heart, X_test_heart, y_train_heart, y_test_heart = preprocess_data(
    heart, "target", "Models/heart_scaler.pkl", "Models/heart_encoders.pkl"
)
X_train_parkinsons, X_test_parkinsons, y_train_parkinsons, y_test_parkinsons = preprocess_data(
    parkinsons, "status", "Models/parkinsons_scaler.pkl", "Models/parkinsons_encoders.pkl"
)

print("Preprocessing complete and scalers saved!")


Preprocessing complete and scalers saved!


In [6]:
import joblib

# Save trained models
# Save trained models
joblib.dump(model_diabetes, 'Models/diabetes_model.pkl')
joblib.dump(model_heart, 'Models/heart_model.pkl')
joblib.dump(model_parkinsons, 'Models/parkinsons_model.pkl')

print("Models saved successfully!")

Models saved successfully!


In [ ]:
import numpy as np
import pandas as pd   #TESTTSTS
import joblib

# Load model and scaler
model_diabetes = joblib.load('Models/diabetes_model.pkl')
scaler_diabetes = joblib.load('Models/diabetes_scaler.pkl')

# Define feature names (use the same column names from training data)
feature_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
                 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

# Test with non-diabetic values
test_input = np.array([[0, 85, 70, 20, 80, 22.0, 0.2, 25]])  # Example input

# Convert to DataFrame to keep column names
test_input_df = pd.DataFrame(test_input, columns=feature_names)

# Scale the test input
test_input_scaled = scaler_diabetes.transform(test_input_df)  # Now using DataFrame

# Make prediction
prediction = model_diabetes.predict(test_input_scaled)

print("Prediction:", prediction)  # Now it should work without the warning


Prediction: [0]


In [13]:
# Check feature names in the dataset
import joblib

scaler_parkinsons = joblib.load('Models/parkinsons_scaler.pkl')

# Get expected feature names
expected_features = scaler_parkinsons.feature_names_in_
print("Expected Feature Names:", expected_features)


Expected Feature Names: ['name' 'MDVP:Fo(Hz)' 'MDVP:Fhi(Hz)' 'MDVP:Flo(Hz)' 'MDVP:Jitter(%)'
 'MDVP:Jitter(Abs)' 'MDVP:RAP' 'MDVP:PPQ' 'Jitter:DDP' 'MDVP:Shimmer'
 'MDVP:Shimmer(dB)' 'Shimmer:APQ3' 'Shimmer:APQ5' 'MDVP:APQ' 'Shimmer:DDA'
 'NHR' 'HNR' 'RPDE' 'DFA' 'spread1' 'spread2' 'D2' 'PPE']
